### OSM WD consistency och skapa karta vindskydd grillplatser

* Issue [#185](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/185)
* [this Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/Notebook/185_WD_OSM.ipynb)

* [SAT Dashboard](https://salgo60.github.io/Stockholm_Archipelago_Trail/Notebook/output/dashboard.html)
   * [karta](https://salgo60.github.io/Stockholm_Archipelago_Trail/Notebook/output/182_WD_OSM_signs_latest.html) 


In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-11-05 07:19


In [2]:
#!/usr/bin/env python3
"""
Generisk OSM↔Wikidata-koll via valfri SPARQL

Syfte
------
Skicka in *vilken SPARQL-fråga som helst* som returnerar ett ?item (Wikidata-objekt)
och **någon** form av OSM-id (node/way/relation) – t.ex. via standardegenskaperna
P10689 (node), P11693 (way), P402 (relation) eller som färdiga URL:er – så kontrollerar
skriptet att **motsvarande OSM-objekt pekar tillbaka** till Wikidata via `wikidata=Q…`.

Vad gör skriptet?
-----------------
1) Kör din SPARQL, tolkar resultatet och extraherar:
   - QID (från ?item)
   - OSM-ID:n (node/way/relation) från valfria fält/format.
2) Hämtar nämnda OSM-objekt direkt via Overpass (per id, ej bbox).
3) Validerar back-link:
   - ✅ **OK**: OSM har `wikidata`-tagg som *innehåller* samma QID (hanterar även semikolon-separerade listor).
   - ⚠️ **SAKNAS**: OSM saknar `wikidata`-tag helt.
   - ❌ **FEL**: OSM har `wikidata`, men den matchar **inte** QID.
4) Skriver ut **listor med klickbara länkar** för varje kategori och exporterar CSV.

Exempelanrop
------------
- `run_generic(SPARQL_QUERY)`

Notera
------
- Du kan fortfarande använda den tidigare `run()` för den hårdkodade SAT-frågan om du vill.
"""
from __future__ import annotations
import re
import math
import json
from typing import Dict, Iterable, List, Tuple

import pandas as pd
import requests

WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
OVERPASS_API = "https://overpass-api.de/api/interpreter"
USER_AGENT = "SAT-OSM-WD-Consistency/2.0 (contact: your-email@example.com)"

# -------------------------
# Hjälpfunktioner
# -------------------------

def make_headers() -> Dict[str, str]:
    return {"User-Agent": USER_AGENT}


def to_osm_prefixed_id(typ: str, osm_id: int) -> str:
    return {"node": "n", "way": "w", "relation": "r"}[typ] + str(int(osm_id))


def parse_osm_from_value(val: str) -> Tuple[str, int] | None:
    """Försök tolka ett OSM-id (typ, id) från godtycklig sträng/URL/tal.
    Stödjer t.ex. 12345, "12345", "https://www.openstreetmap.org/node/12345".
    Returnerar (typ, id) eller None.
    """
    if val is None:
        return None
    s = str(val).strip()

    # URL-format
    m = re.search(r"openstreetmap\.org\/(node|way|relation)\/(\d+)", s)
    if m:
        return m.group(1), int(m.group(2))

    # Rena tal: lämna typ okänd
    if s.isdigit():
        # Typ avgörs av fältnamnet – hanteras av anroparen
        return ("unknown", int(s))

    return None


# -------------------------
# Wikidata
# -------------------------

def fetch_wikidata_generic(sparql: str) -> pd.DataFrame:
    """Kör godtycklig SPARQL och extraherar QID och OSM-id:n ur valfria kolumner.
    Krav: ?item måste finnas och vara ett WD-objekt.
    Heuristik för OSM:
      - Leta kolumnnamn som innehåller något av: node|way|rel|osm|OSM
      - För varje sådan kolumn: tolka värdet som URL eller siffra.
      - Om typ är "unknown" – gissa typ utifrån kolumnnamn (node/way/relation).
    Returnerar DF med kolumner: qid, label (om finns), osm_type, osm_id
    (en rad per QID×OSM-referens). Om ingen OSM-referens hittas för en QID
    returneras ändå en rad utan OSM-typ/id för spårbarhet.
    """
    r = requests.get(WIKIDATA_SPARQL, params={"query": sparql, "format": "json"}, headers=make_headers())
    r.raise_for_status()
    rows = r.json()["results"]["bindings"]

    out_rows: List[Dict] = []
    for b in rows:
        item_uri = b.get("item", {}).get("value")
        if not item_uri:
            continue
        qid = item_uri.rsplit("/", 1)[-1]
        label = b.get("itemLabel", {}).get("value")

        # samla alla kandidater till OSM-kolumner
        osm_keys = [k for k in b.keys() if re.search(r"(node|way|rel|osm)", k, re.I) and k != "item"]

        found_any = False
        for k in osm_keys:
            raw = b[k].get("value")
            parsed = parse_osm_from_value(raw)
            if parsed:
                typ, oid = parsed
                if typ == "unknown":
                    # gissa från kolumnnamn
                    lk = k.lower()
                    if "node" in lk:
                        typ = "node"
                    elif "way" in lk:
                        typ = "way"
                    elif "rel" in lk:
                        typ = "relation"
                    else:
                        # kan ej avgöra – hoppa
                        continue
                out_rows.append({
                    "qid": qid, "label": label, "osm_type": typ, "osm_id": int(oid),
                    "osm_url": f"https://www.openstreetmap.org/{typ}/{int(oid)}",
                })
                found_any = True

        if not found_any:
            # lägg en rad utan OSM för spårbarhet
            out_rows.append({"qid": qid, "label": label, "osm_type": None, "osm_id": None, "osm_url": None})

    df = pd.DataFrame(out_rows).drop_duplicates()
    return df


# -------------------------
# Overpass
# -------------------------

def fetch_osm_by_ids(df_refs: pd.DataFrame) -> pd.DataFrame:
    """Hämta OSM-objekt per id för de rader som har osm_type/osm_id.
    Returnerar kolumner: osm_type, osm_id, url, wikidata_tag
    """
    subset = df_refs.dropna(subset=["osm_type", "osm_id"]).copy()
    if subset.empty:
        return pd.DataFrame(columns=["osm_type","osm_id","url","wikidata_tag"])   

    ids_by_type: Dict[str, List[int]] = {"node": [], "way": [], "relation": []}
    for _, r in subset.iterrows():
        t = r["osm_type"]
        if t in ids_by_type:
            ids_by_type[t].append(int(r["osm_id"]))

    def batched(lst: List[int], n: int = 200) -> Iterable[List[int]]:
        for i in range(0, len(lst), n):
            yield lst[i:i+n]

    rows: List[Dict] = []
    for t in ("node","way","relation"):
        if not ids_by_type[t]:
            continue
        for batch in batched(sorted(set(ids_by_type[t]))):
            ids_str = ",".join(str(i) for i in batch)
            q = f"""
            [out:json][timeout:50];
            {t}(id:{ids_str});
            out ids tags center;
            """.strip()
            rr = requests.post(OVERPASS_API, data={"data": q}, headers=make_headers())
            rr.raise_for_status()
            for el in rr.json().get("elements", []):
                tags = el.get("tags", {})
                rows.append({
                    "osm_type": t,
                    "osm_id": el.get("id"),
                    "url": f"https://www.openstreetmap.org/{t}/{el.get('id')}",
                    "wikidata_tag": tags.get("wikidata"),
                    "name": tags.get("name"),
                })

    return pd.DataFrame(rows)


# -------------------------
# Validering & utskrift
# -------------------------

def normalize_wikidata_values(val: str | None) -> List[str]:
    if not val:
        return []
    # Dela på semikolon/komma och trimma, ta bara sådant som ser ut som Q…
    parts = re.split(r"[;|,]", val)
    return [p.strip() for p in parts if re.match(r"^Q\d+$", p.strip(), re.I)]


def verify_backlinks(df_refs: pd.DataFrame, df_osm: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    # join på (osm_type, osm_id)
    key = ["osm_type","osm_id"]
    merged = pd.merge(df_refs.dropna(subset=key), df_osm, on=key, how="left", suffixes=("","_osm"))

    ok_rows = []
    missing_rows = []
    wrong_rows = []

    for _, r in merged.iterrows():
        qid = r["qid"]
        tag = r.get("wikidata_tag")
        if pd.isna(tag) or not tag:
            missing_rows.append(r)
        else:
            vals = normalize_wikidata_values(tag)
            if qid in vals:
                ok_rows.append(r)
            else:
                wrong_rows.append(r)

    return {
        "ok": pd.DataFrame(ok_rows),
        "missing": pd.DataFrame(missing_rows),
        "wrong": pd.DataFrame(wrong_rows),
        "all_refs": df_refs,
        "all_osm": df_osm,
    }


def print_link_lists(res: Dict[str, pd.DataFrame]):
    def print_group(title: str, df: pd.DataFrame):
        print(f"\n{title}")
        print("-" * len(title))
        if df.empty:
            print("(inget)")
            return
        # Grupp: per QID
        for qid, grp in df.groupby("qid"):
            print(f"{qid}:")
            for _, r in grp.iterrows():
                print(f"  {r['url']}")

    print_group("❌ OSM har wikidata men pekar fel", res["wrong"])        
    print_group("✅ OSM back-link OK (wikidata matchar QID)", res["ok"])    
    print_group("⚠️ OSM saknar wikidata-tag (lägg till QID)", res["missing"]) 


# -------------------------
# Publika körfunktioner
# -------------------------

def run_generic(sparql_query: str, export_prefix: str = "wd_osm_backlinks"):
    print("Kör SPARQL…")
    refs = fetch_wikidata_generic(sparql_query)
    print(f"Rader från SPARQL (QID×OSM-ref): {len(refs)}")

    print("Hämtar OSM per id…")
    osm = fetch_osm_by_ids(refs)
    print(f"OSM-objekt hämtade: {len(osm)}")

    res = verify_backlinks(refs, osm)

    # Exportera CSV
    for name, df in res.items():
        path = f"{export_prefix}_{name}.csv"
        df.to_csv(path, index=False)
        print(f"↳ sparade {path} ({len(df)} rader)")

    # Listrapport med klickbara länkar
    print_link_lists(res)



def run_sat_example():
    run_generic(SAT_SPARQL, export_prefix="sat_backlinks")


In [6]:
from string import Template
from datetime import datetime as dt
import html  # vi använder html.escape här

def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
    offset_px=(10, 54),  # (top, left) – 54px för att hamna snällt bredvid zoom
):
    """Superenkel, robust About-box som alltid syns."""
    if created_date is None:
        created_date = dt.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"
    top, left  = offset_px
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    links = [
        ("SAT Dashboard", "dashboard.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {html.escape(t)}</a></div>'
        for t, u in links
    )

    tpl = Template(r"""
<style>
  .sat-about {
    position: fixed; z-index: 10000;
    top: ${top}px; left: ${left}px;
    background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    min-width: 240px; max-width: 320px; pointer-events: auto;
  }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
</style>

<div id="${box_id}" class="sat-about ${collapsed_class}">
  <div id="${header_id}" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span><span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="${issue_url}" target="_blank">#${issue_number}</a>&nbsp;&nbsp; Map: ${map_name}</div>
    <div>Created: ${created_date}</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">${links_html}</div>
  </div>
</div>

<script>
(function(){
  var boxId = "${box_id}";
  var hdrId = "${header_id}";
  var storageKey = "satAboutCollapsed_${map_dom_id}_#${issue_number}";

  function setCollapsed(box, collapsed) {
    if (!box) return;
    if (collapsed) box.classList.add("sat-about-collapsed");
    else box.classList.remove("sat-about-collapsed");
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function init(){
    var box = document.getElementById(boxId);
    var hdr = document.getElementById(hdrId);
    if (!box || !hdr) return;

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      setCollapsed(box, !box.classList.contains("sat-about-collapsed"));
    });
  }

  if (document.readyState === "loading") {
    document.addEventListener("DOMContentLoaded", init);
  } else {
    init();
  }
})();
</script>
""")

    html_code = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        issue_number=issue_number,
        issue_url=issue_url,
        map_name=html.escape(map_name),
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
        top=top, left=left,
    )
    m.get_root().html.add_child(folium.Element(html_code))

In [7]:
import os, json, datetime, html, urllib.parse
import requests
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap, MeasureControl, MousePosition

# =====================================
# KONFIG
# =====================================
OUTPUT_PREFIX      = "SAT185_vindskydd_enkel"
FETCH_FROM_WD      = True                 # sätt False om du redan har CSV lokalt
OUT_CSV            = "vindskydd_wd.csv"

# enkla centreringar för externa sajter
NATURKARTAN_TMPL   = "https://www.naturkartan.se/sv/map?lat={lat:.6f}&lon={lon:.6f}&z=14"
GRILLPLATSER_TMPL  = "https://grillplatser.nu/#15/{lat:.6f}/{lon:.6f}"

# =====================================
# SPARQL: Vindskydd (Q1546788) och Grillplats (Q1797440) på SAT (Q134294510),
#         med koordinat, bild (P18) och ev. officiell webb (P856)
# =====================================
SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?website WHERE {
  VALUES ?type { wd:Q1546788 wd:Q1797440 }
  ?item wdt:P31 ?type .            # instans av vindskydd / grillplats
  ?item wdt:P6104 wd:Q134294510 .  # kopplad till Stockholm Archipelago Trail
  ?item wdt:P625 ?coord .          # koordinat
  OPTIONAL { ?item wdt:P18 ?img }  # bild
  OPTIONAL { ?item wdt:P856 ?website } # ev. officiell webbplats
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,mul" }
}
"""

# =====================================
# HJÄLPARE
# =====================================
def fetch_wd_to_csv(out_csv=OUT_CSV):
    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "SAT-map-builder/1.2 (contact: salgo60@msn.com)"
    }
    r = requests.get(url, params={"query": SPARQL}, headers=headers, timeout=80)
    r.raise_for_status()
    data = r.json()
    rows = []
    for b in data["results"]["bindings"]:
        gv = lambda k: b[k]["value"] if k in b else None
        rows.append({
            "item": gv("item"),
            "itemLabel": gv("itemLabel"),
            "coord": gv("coord"),
            "img": gv("img"),
            "website": gv("website"),
        })
    df = pd.DataFrame(rows)
    # parse coord "Point(lon lat)"
    lats, lons = [], []
    for v in df["coord"].astype(str):
        lat = lon = None
        if v.startswith("Point(") and v.endswith(")"):
            parts = v[6:-1].split()
            if len(parts) == 2:
                try:
                    lon = float(parts[0]); lat = float(parts[1])
                except Exception:
                    pass
        lats.append(lat); lons.append(lon)
    df["lat"], df["lon"] = lats, lons
    df.to_csv(out_csv, index=False)
    return df

def commons_filepage_from_p18(p18_url: str | None) -> str | None:
    """Försök att bygga en Commons-fil-sida från P18-URL (kan vara FilePath-länk)."""
    if not p18_url:
        return None
    # Om det redan är en valbar commons-sida, returnera
    if "commons.wikimedia.org/wiki/File:" in p18_url:
        return p18_url
    # Försök extrahera filnamnet ur ev. FilePath-URL
    try:
        fname = p18_url.split("/")[-1]
        if fname:
            return "https://commons.wikimedia.org/wiki/File:" + urllib.parse.unquote(fname)
    except Exception:
        pass
    return p18_url

# =====================================
# 1) Hämta/läs från Wikidata
# =====================================
if FETCH_FROM_WD:
    print("Hämtar från Wikidata…")
    df = fetch_wd_to_csv(OUT_CSV)
else:
    if not os.path.exists(OUT_CSV):
        raise FileNotFoundError(f"Saknar {OUT_CSV}. Sätt FETCH_FROM_WD=True första gången.")
    df = pd.read_csv(OUT_CSV)

# Städa bort rader utan koordinat
df = df.dropna(subset=["lat", "lon"]).copy()
print(f"Antal objekt: {len(df)}")

# =====================================
# 2) Initiera karta (enkel)
# =====================================
if len(df) == 0:
    # fallback Stockholm
    center = [59.3293, 18.0686]
else:
    center = [df["lat"].mean(), df["lon"].mean()]

m = folium.Map(location=center, zoom_start=10, tiles="CartoDB Positron", control_scale=True)
Fullscreen().add_to(m)
MiniMap(toggle_display=True, position="bottomleft").add_to(m)
MeasureControl(position='topleft', primary_length_unit='kilometers', secondary_length_unit='meters').add_to(m)
MousePosition(position='bottomright', separator=' | ', num_digits=5, prefix='Lat/Lon:').add_to(m)

# =====================================
# 3) Markörer + Popups enligt önskemål
#     * Bild (P18)
#     * Länk: grillplatser.nu (centrerad på koordinaten)
#     * Länk: Naturkartan (centrerad på koordinaten)
#     * Länk: Wikimedia Commons (fil-sida från P18)
#     * Länk: Officiell webb (P856) – men bara om den inte är grillplatser.nu
# =====================================
fg = folium.FeatureGroup(name="Vindskydd/Grillplatser på SAT", show=True)

for _, row in df.iterrows():
    lat, lon = float(row["lat"]), float(row["lon"])
    label    = str(row.get("itemLabel") or "Objekt")
    item_uri = str(row.get("item") or "")
    p18      = row.get("img")
    website  = row.get("website")

    # Länkar
    wd_link  = item_uri if item_uri.startswith("http") else None
    commons  = commons_filepage_from_p18(p18)
    natur    = NATURKARTAN_TMPL.format(lat=lat, lon=lon)
    grill    = GRILLPLATSER_TMPL.format(lat=lat, lon=lon)

    # Officiell webb – exkludera om det ÄR grillplatser.nu
    website_ok = None
    if isinstance(website, str) and website.strip():
        if "grillplatser.nu" not in website.lower():
            website_ok = website

    # Bild
    img_html = ""
    if isinstance(p18, str) and p18:
        img_html = f"""
        <div style=\"margin:8px 0;\">
          <div style=\"
            width:100%; max-width:320px; aspect-ratio: 4 / 3; overflow:hidden;
            border-radius:10px; box-shadow:0 6px 18px rgba(0,0,0,.18);
          \">
            <img src=\"{html.escape(p18)}\" style=\"width:100%; height:100%; object-fit:cover;\" referrerpolicy=\"no-referrer\">
          </div>
        </div>
        """

    # Lista med önskade länkar
    links = []
    links.append(f"🔥 <a href='{html.escape(grill)}' target='_blank'>grillplatser.nu (karta)</a>")
    links.append(f"🌿 <a href='{html.escape(natur)}' target='_blank'>Naturkartan (karta)</a>")
    if commons:
        links.append(f"🖼️ <a href='{html.escape(commons)}' target='_blank'>Wikimedia Commons (P18)</a>")
    if website_ok:
        links.append(f"🌐 <a href='{html.escape(website_ok)}' target='_blank'>Officiell webb</a>")
    if wd_link:
        links.append(f"🧠 <a href='{html.escape(wd_link)}' target='_blank'>Wikidata</a>")

    links_html = "<ul style='list-style:none; padding:0; margin:6px 0 0 0; font-size:13px; line-height:1.4;'>" + \
                 "".join(f"<li style='margin:2px 0;'>{l}</li>" for l in links) + "</ul>"

    html_popup = f"""
    <div style=\"font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; max-width: 340px; line-height:1.35;\">
      <div style=\"font-weight:600; font-size:15px; margin-bottom:4px;\">{html.escape(label)}</div>
      {img_html}
      {links_html}
    </div>
    """

    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(icon="info-sign", color="green"),
        popup=folium.Popup(html_popup, max_width=380)
    ).add_to(fg)

fg.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# =====================================
# 4) Spara
# =====================================
os.makedirs("output", exist_ok=True)

ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")

m.save(out_ts)
m.save(out_latest)

print("Klar:")
print("  ", out_ts)
print("  ", out_latest)


Hämtar från Wikidata…
Antal objekt: 86
Klar:
   output/SAT185_vindskydd_enkel_20251105_072202.html
   output/SAT185_vindskydd_enkel_latest.html


In [8]:
import os, json, datetime, html, urllib.parse
import requests
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap, MeasureControl, MousePosition

# =====================================
# KONFIG
# =====================================
OUTPUT_PREFIX      = "SAT185_vindskydd_enkel"
FETCH_FROM_WD      = True                 # sätt False om du redan har CSV lokalt
OUT_CSV            = "vindskydd_wd.csv"

# enkla centreringar för externa sajter
NATURKARTAN_TMPL   = "https://www.naturkartan.se/sv/map?lat={lat:.6f}&lon={lon:.6f}&z=14"
GRILLPLATSER_TMPL  = "https://grillplatser.nu/#15/{lat:.6f}/{lon:.6f}"

# =====================================
# SPARQL: Vindskydd (Q1546788) och Grillplats (Q1797440) på SAT (Q134294510),
#         med koordinat, bild (P18) och ev. officiell webb (P856)
# =====================================
SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?website WHERE {
  VALUES ?type { wd:Q1546788 wd:Q1797440 }
  ?item wdt:P31 ?type .            # instans av vindskydd / grillplats
  ?item wdt:P6104 wd:Q134294510 .  # kopplad till Stockholm Archipelago Trail
  ?item wdt:P625 ?coord .          # koordinat
  OPTIONAL { ?item wdt:P18 ?img }  # bild
  OPTIONAL { ?item wdt:P856 ?website } # ev. officiell webbplats
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,mul" }
}
"""

# =====================================
# HJÄLPARE
# =====================================
def fetch_wd_to_csv(out_csv=OUT_CSV):
    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "SAT-map-builder/1.2 (contact: salgo60@msn.com)"
    }
    r = requests.get(url, params={"query": SPARQL}, headers=headers, timeout=80)
    r.raise_for_status()
    data = r.json()
    rows = []
    for b in data["results"]["bindings"]:
        gv = lambda k: b[k]["value"] if k in b else None
        rows.append({
            "item": gv("item"),
            "itemLabel": gv("itemLabel"),
            "coord": gv("coord"),
            "img": gv("img"),
            "website": gv("website"),
        })
    df = pd.DataFrame(rows)
    # parse coord "Point(lon lat)"
    lats, lons = [], []
    for v in df["coord"].astype(str):
        lat = lon = None
        if v.startswith("Point(") and v.endswith(")"):
            parts = v[6:-1].split()
            if len(parts) == 2:
                try:
                    lon = float(parts[0]); lat = float(parts[1])
                except Exception:
                    pass
        lats.append(lat); lons.append(lon)
    df["lat"], df["lon"] = lats, lons
    df.to_csv(out_csv, index=False)
    return df

def commons_filepage_from_p18(p18_url: str | None) -> str | None:
    """Försök att bygga en Commons-fil-sida från P18-URL (kan vara FilePath-länk)."""
    if not p18_url:
        return None
    # Om det redan är en valbar commons-sida, returnera
    if "commons.wikimedia.org/wiki/File:" in p18_url:
        return p18_url
    # Försök extrahera filnamnet ur ev. FilePath-URL
    try:
        fname = p18_url.split("/")[-1]
        if fname:
            return "https://commons.wikimedia.org/wiki/File:" + urllib.parse.unquote(fname)
    except Exception:
        pass
    return p18_url

# =====================================
# 1) Hämta/läs från Wikidata
# =====================================
if FETCH_FROM_WD:
    print("Hämtar från Wikidata…")
    df = fetch_wd_to_csv(OUT_CSV)
else:
    if not os.path.exists(OUT_CSV):
        raise FileNotFoundError(f"Saknar {OUT_CSV}. Sätt FETCH_FROM_WD=True första gången.")
    df = pd.read_csv(OUT_CSV)

# Städa bort rader utan koordinat
df = df.dropna(subset=["lat", "lon"]).copy()
print(f"Antal objekt: {len(df)}")

# =====================================
# 2) Initiera karta (enkel)
# =====================================
if len(df) == 0:
    # fallback Stockholm
    center = [59.3293, 18.0686]
else:
    center = [df["lat"].mean(), df["lon"].mean()]

m = folium.Map(location=center, zoom_start=10, tiles="CartoDB Positron", control_scale=True)
Fullscreen().add_to(m)
MiniMap(toggle_display=True, position="bottomleft").add_to(m)
MeasureControl(position='topleft', primary_length_unit='kilometers', secondary_length_unit='meters').add_to(m)
MousePosition(position='bottomright', separator=' | ', num_digits=5, prefix='Lat/Lon:').add_to(m)

# =====================================
# 3) Markörer + Popups enligt önskemål
#     * Bild (P18)
#     * Länk: grillplatser.nu (centrerad på koordinaten)
#     * Länk: Naturkartan (centrerad på koordinaten)
#     * Länk: Wikimedia Commons (fil-sida från P18)
#     * Länk: Officiell webb (P856) – men bara om den inte är grillplatser.nu
# =====================================
fg = folium.FeatureGroup(name="Vindskydd/Grillplatser på SAT", show=True)

for _, row in df.iterrows():
    lat, lon = float(row["lat"]), float(row["lon"])
    label    = str(row.get("itemLabel") or "Objekt")
    item_uri = str(row.get("item") or "")
    p18      = row.get("img")
    website  = row.get("website")

    # Länkar
    wd_link  = item_uri if item_uri.startswith("http") else None
    commons  = commons_filepage_from_p18(p18)
    natur    = NATURKARTAN_TMPL.format(lat=lat, lon=lon)
    grill    = GRILLPLATSER_TMPL.format(lat=lat, lon=lon)

    # Officiell webb – exkludera om det ÄR grillplatser.nu
    website_ok = None
    if isinstance(website, str) and website.strip():
        if "grillplatser.nu" not in website.lower():
            website_ok = website

    # Bild
    img_html = ""
    if isinstance(p18, str) and p18:
        img_html = f"""
        <div style=\"margin:8px 0;\">
          <div style=\"
            width:100%; max-width:320px; aspect-ratio: 4 / 3; overflow:hidden;
            border-radius:10px; box-shadow:0 6px 18px rgba(0,0,0,.18);
          \">
            <img src=\"{html.escape(p18)}\" style=\"width:100%; height:100%; object-fit:cover;\" referrerpolicy=\"no-referrer\">
          </div>
        </div>
        """

    # Lista med önskade länkar
    links = []
    links.append(f"🔥 <a href='{html.escape(grill)}' target='_blank'>grillplatser.nu (karta)</a>")
    links.append(f"🌿 <a href='{html.escape(natur)}' target='_blank'>Naturkartan (karta)</a>")
    if commons:
        links.append(f"🖼️ <a href='{html.escape(commons)}' target='_blank'>Wikimedia Commons (P18)</a>")
    if website_ok:
        links.append(f"🌐 <a href='{html.escape(website_ok)}' target='_blank'>Officiell webb</a>")
    if wd_link:
        links.append(f"🧠 <a href='{html.escape(wd_link)}' target='_blank'>Wikidata</a>")

    links_html = "<ul style='list-style:none; padding:0; margin:6px 0 0 0; font-size:13px; line-height:1.4;'>" + \
                 "".join(f"<li style='margin:2px 0;'>{l}</li>" for l in links) + "</ul>"

    html_popup = f"""
    <div style=\"font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; max-width: 340px; line-height:1.35;\">
      <div style=\"font-weight:600; font-size:15px; margin-bottom:4px;\">{html.escape(label)}</div>
      {img_html}
      {links_html}
    </div>
    """

    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(icon="info-sign", color="green"),
        popup=folium.Popup(html_popup, max_width=380)
    ).add_to(fg)

fg.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# =====================================
# 4) Spara
# =====================================
os.makedirs("output", exist_ok=True)

ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")

m.save(out_ts)
m.save(out_latest)

print("Klar:")
print("  ", out_ts)
print("  ", out_latest)


Hämtar från Wikidata…
Antal objekt: 86
Klar:
   output/SAT185_vindskydd_enkel_20251105_072202.html
   output/SAT185_vindskydd_enkel_latest.html


In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Vindskydd & grillplatser (SAT) → Wikidata → Folium-karta
- OSM-länkar hämtas direkt från Wikidata (P402, P10689/P11693, P8189).
- SAT-linjen läses från lokal GeoJSON: SAT_full.geojson (om finns).

Krav:
    pip install pandas folium requests SPARQLWrapper

Kör:
    python vindskydd_grillplatser_map.py

Utdata:
    map_vindskydd_grillplatser.html
"""

import json
from pathlib import Path
from typing import List, Optional

import pandas as pd
import folium
from SPARQLWrapper import SPARQLWrapper, JSON

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

# Sökvägar för SAT-geojson
SAT_PATHS = [Path("SAT_full.geojson"), Path("/mnt/data/SAT_full.geojson")]

SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?website ?naturkartanURL ?commonsURL373 ?commonsURL3608 ?grillplatsnuref
       ?osmRelURL ?osmWayURL ?osmNodeURL WHERE {
  VALUES ?type { wd:Q1546788 wd:Q1797440 }        # vindskydd, grillplats
  ?item wdt:P31 ?type .
  ?item wdt:P6104 wd:Q134294510 .                 # kopplad till Stockholm Archipelago Trail
  ?item wdt:P625 ?coord .

  OPTIONAL { ?item wdt:P18 ?img }
  OPTIONAL { ?item wdt:P856 ?website }

  OPTIONAL {
    ?item wdt:P10467 ?naturkartan .
    BIND( IRI(CONCAT("https://www.naturkartan.se/", STR(?naturkartan))) AS ?naturkartanURL )
  }
  OPTIONAL {
    ?item wdt:P373 ?commons373 .
    BIND( IRI(CONCAT("https://commons.wikimedia.org/wiki/Category:", ENCODE_FOR_URI(STR(?commons373)))) AS ?commonsURL373 )
  }
  OPTIONAL {
    ?item wdt:P3608 ?commons3608 .
    BIND( IRI(CONCAT("https://commons.wikimedia.org/wiki/Category:", ENCODE_FOR_URI(STR(?commons3608)))) AS ?commonsURL3608 )
  }
  OPTIONAL {
    ?item wdt:P1343 wd:Q120778083 .
    ?item p:P1343 ?statement .
    ?statement prov:wasDerivedFrom ?ref .
    ?ref pr:P854 ?grillplatsnuref .
  }

  OPTIONAL {
    ?item wdt:P402 ?osmRel .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/relation/", STR(?osmRel))) AS ?osmRelURL )
  }
  OPTIONAL {
    ?item wdt:P10689 ?osmWay10689 .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/way/", STR(?osmWay10689))) AS ?osmWayURL )
  }
  OPTIONAL {
    ?item wdt:P11693 ?osmWay11693 .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/way/", STR(?osmWay11693))) AS ?osmWayURL )
  }
  OPTIONAL {
    ?item wdt:P8189 ?osmNode .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/node/", STR(?osmNode))) AS ?osmNodeURL )
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,mul". }
}
"""

def run_sparql() -> pd.DataFrame:
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT, agent="SAT-vindskydd-map/2.0")
    sparql.setQuery(SPARQL)
    sparql.setReturnFormat(JSON)
    data = sparql.query().convert()
    rows = []
    for b in data["results"]["bindings"]:
        def val(x, f="value"):
            return b[x][f] if x in b else None
        coord = val("coord")
        lat, lon = None, None
        if coord:
            try:
                inside = coord.split("(")[1].split(")")[0].strip()
                parts = inside.split(" ")
                lon = float(parts[0]); lat = float(parts[1])
            except Exception:
                pass
        rows.append({
            "item": val("item"),
            "itemLabel": val("itemLabel"),
            "lat": lat,
            "lon": lon,
            "img": val("img"),
            "website": val("website"),
            "naturkartanURL": val("naturkartanURL"),
            "commonsURL373": val("commonsURL373"),
            "commonsURL3608": val("commonsURL3608"),
            "grillplatsnuref": val("grillplatsnuref"),
            "osmRelURL": val("osmRelURL"),
            "osmWayURL": val("osmWayURL"),
            "osmNodeURL": val("osmNodeURL"),
        })
    df = pd.DataFrame(rows)
    if df.empty:
        return df

    agg = {
        'itemLabel': 'first',
        'lat': 'first', 'lon': 'first', 'img': 'first', 'website': 'first',
        'naturkartanURL': 'first', 'grillplatsnuref': 'first',
        'commonsURL373': lambda s: list({x for x in s if pd.notna(x)}),
        'commonsURL3608': lambda s: list({x for x in s if pd.notna(x)}),
        'osmRelURL': lambda s: list({x for x in s if pd.notna(x)}),
        'osmWayURL': lambda s: list({x for x in s if pd.notna(x)}),
        'osmNodeURL': lambda s: list({x for x in s if pd.notna(x)}),
    }
    g = df.groupby('item', as_index=False).agg(agg)
    g['commonsURLS'] = g.apply(lambda r: list(dict.fromkeys([*(r['commonsURL373'] or []), *(r['commonsURL3608'] or [])])), axis=1)
    def _merge_osm(r):
        lst = [*(r['osmRelURL'] or []), *(r['osmWayURL'] or []), *(r['osmNodeURL'] or [])]
        seen, out = set(), []
        for u in lst:
            if u not in seen:
                seen.add(u); out.append(u)
        return out
    g['osmLinks'] = g.apply(_merge_osm, axis=1)
    return g

def build_popup_html(row: pd.Series) -> str:
    website = row.get("website")
    if website and "grillplatser.nu" in website:
        website = None
    img_html = ""
    if row.get("img"):
        img_html = f'<div><img src="{row["img"]}" style="max-width:240px; border-radius:8px;"></div>'
    links: List[str] = []
    if website:
        links.append(f'<a href="{website}" target="_blank">Webbplats</a>')
    if row.get("naturkartanURL"):
        links.append(f'<a href="{row["naturkartanURL"]}" target="_blank">Naturkartan</a>')
    for idx, u in enumerate(row.get("commonsURLS") or [], 1):
        label = "Wikimedia Commons" if len(row["commonsURLS"]) == 1 else f"Wikimedia Commons {idx}"
        links.append(f'<a href="{u}" target="_blank">{label}</a>')
    if row.get("grillplatsnuref"):
        links.append(f'<a href="{row["grillplatsnuref"]}" target="_blank">grillplatser.nu</a>')
    for idx, u in enumerate(row.get("osmLinks") or [], 1):
        label = "OSM" if len(row["osmLinks"]) == 1 else f"OSM {idx}"
        links.append(f'<a href="{u}" target="_blank">{label}</a>')
    title = row.get("itemLabel") or "Okänd plats"
    links_html = " · ".join(links) if links else "—"
    return f'<b>{title}</b><br>{img_html}{links_html}'

def add_sat_geojson(m: folium.Map) -> Optional[str]:
    for p in SAT_PATHS:
        if p.exists():
            try:
                gj = folium.GeoJson(p.read_text(encoding="utf-8"), name="SAT",
                    style_function=lambda f: {"weight": 3},
                    highlight_function=lambda f: {"weight": 5})
                gj.add_to(m); return str(p)
            except Exception:
                try:
                    gj = folium.GeoJson(json.loads(p.read_text()), name="SAT")
                    gj.add_to(m); return str(p)
                except Exception: pass
    return None

def main():
    df = run_sparql()
    if df.empty:
        raise SystemExit("Inga resultat från Wikidata.")
    m = folium.Map(location=[df["lat"].mean(), df["lon"].mean()], zoom_start=9, tiles="CartoDB positron")
    add_sat_geojson(m)
    fg_has, fg_none = folium.FeatureGroup("Har OSM-länk"), folium.FeatureGroup("Saknar OSM-länk")
    for _, row in df.iterrows():
        html = build_popup_html(row)
        marker = folium.Marker([row["lat"], row["lon"]],
            popup=folium.Popup(html, max_width=300),
            tooltip=row.get("itemLabel"),
            icon=folium.Icon(color="green" if row.get("osmLinks") else "red"))
        (fg_has if row.get("osmLinks") else fg_none).add_child(marker)
    fg_has.add_to(m); fg_none.add_to(m); folium.LayerControl().add_to(m)
    add_about_box(m, issue_number=188, map_name="SAT vindskydd grillplatser")
    m.save("SAT185_map_vindskydd_grillplatser.html")
    print("saved map")

if __name__ == "__main__":
    main()


saved map


In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Vindskydd & grillplatser (SAT) → Wikidata → Folium-karta
- OSM-länkar hämtas direkt från Wikidata (Relation: P402, Way: P10689, Node: P11693).
- Hämtar även P137 (operator) och färgkodar markörer som har operator = Q131318799 (Stockholm Archipelago Trail).
- Olika ikoner för grillplats (Q1546788) respektive vindskydd (Q1797440).
- SAT-linjen läses från lokal GeoJSON: SAT_full.geojson (om finns).

Krav:
    pip install pandas folium requests SPARQLWrapper

Kör:
    python vindskydd_grillplatser_map.py

Utdata:
    map_vindskydd_grillplatser.html
"""
from datetime import datetime
import html as _html
import html

import json
from pathlib import Path
from typing import List, Optional

import pandas as pd
import folium
from SPARQLWrapper import SPARQLWrapper, JSON

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

# Sökvägar för SAT-geojson
SAT_PATHS = [Path("SAT_full.geojson"), Path("/mnt/data/SAT_full.geojson")]

# Q-id
Q_GRILLPLATS = "http://www.wikidata.org/entity/Q1546788"
Q_VINDSKYDD  = "http://www.wikidata.org/entity/Q1797440"
Q_SAT        = "http://www.wikidata.org/entity/Q131318799"  # Stockholm Archipelago Trail

SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?website ?naturkartanURL ?commonsURL373 ?commonsURL3608 ?grillplatsnuref
       ?osmRelURL ?osmWayURL ?osmNodeURL ?type ?operator WHERE {
  VALUES ?type { wd:Q1546788 wd:Q1797440 }        # grillplats, vindskydd
  ?item wdt:P31 ?type .
  ?item wdt:P6104 wd:Q134294510 .                 # kopplad till Stockholm Archipelago Trail
  ?item wdt:P625 ?coord .

  OPTIONAL { ?item wdt:P18 ?img }
  OPTIONAL { ?item wdt:P856 ?website }
  OPTIONAL { ?item wdt:P137 ?operator }           # operator

  OPTIONAL {
    ?item wdt:P10467 ?naturkartan .
    BIND( IRI(CONCAT("https://www.naturkartan.se/", STR(?naturkartan))) AS ?naturkartanURL )
  }
  OPTIONAL {
    ?item wdt:P373 ?commons373 .
    BIND( IRI(CONCAT("https://commons.wikimedia.org/wiki/Category:", ENCODE_FOR_URI(STR(?commons373)))) AS ?commonsURL373 )
  }
  OPTIONAL {
    ?item wdt:P3608 ?commons3608 .
    BIND( IRI(CONCAT("https://commons.wikimedia.org/wiki/Category:", ENCODE_FOR_URI(STR(?commons3608)))) AS ?commonsURL3608 )
  }
  OPTIONAL {
    ?item wdt:P1343 wd:Q120778083 .
    ?item p:P1343 ?statement .
    ?statement prov:wasDerivedFrom ?ref .
    ?ref pr:P854 ?grillplatsnuref .
  }

  # OSM-kopplingar (relation, way, node)
  OPTIONAL {
    ?item wdt:P402 ?osmRel .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/relation/", STR(?osmRel))) AS ?osmRelURL )
  }
  OPTIONAL {
    ?item wdt:P10689 ?osmWay .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/way/", STR(?osmWay))) AS ?osmWayURL )
  }
  OPTIONAL {
    ?item wdt:P11693 ?osmNode .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/node/", STR(?osmNode))) AS ?osmNodeURL )
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,mul". }
}
"""

def run_sparql() -> pd.DataFrame:
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT, agent="SAT-vindskydd-map/2.2")
    sparql.setQuery(SPARQL)
    sparql.setReturnFormat(JSON)
    data = sparql.query().convert()
    rows = []
    for b in data["results"]["bindings"]:
        def val(x, f="value"):
            return b[x][f] if x in b else None
        coord = val("coord")
        lat, lon = None, None
        if coord:
            try:
                inside = coord.split("(")[1].split(")")[0].strip()
                parts = inside.split(" ")
                lon = float(parts[0]); lat = float(parts[1])
            except Exception:
                pass
        rows.append({
            "item": val("item"),
            "itemLabel": val("itemLabel"),
            "lat": lat, "lon": lon,
            "img": val("img"),
            "website": val("website"),
            "naturkartanURL": val("naturkartanURL"),
            "commonsURL373": val("commonsURL373"),
            "commonsURL3608": val("commonsURL3608"),
            "grillplatsnuref": val("grillplatsnuref"),
            "osmRelURL": val("osmRelURL"),
            "osmWayURL": val("osmWayURL"),
            "osmNodeURL": val("osmNodeURL"),
            "type": val("type"),
            "operator": val("operator"),
        })
    df = pd.DataFrame(rows)
    if df.empty:
        return df

    agg = {
        'itemLabel': 'first', 'lat': 'first', 'lon': 'first',
        'img': 'first', 'website': 'first', 'naturkartanURL': 'first', 'grillplatsnuref': 'first',
        'commonsURL373': lambda s: list({x for x in s if pd.notna(x)}),
        'commonsURL3608': lambda s: list({x for x in s if pd.notna(x)}),
        'osmRelURL': lambda s: list({x for x in s if pd.notna(x)}),
        'osmWayURL': lambda s: list({x for x in s if pd.notna(x)}),
        'osmNodeURL': lambda s: list({x for x in s if pd.notna(x)}),
        'type': 'first',
        'operator': lambda s: list({x for x in s if pd.notna(x)}),
    }
    g = df.groupby('item', as_index=False).agg(agg)

    # Commons-länkar
    g['commonsURLS'] = g.apply(
        lambda r: list(dict.fromkeys([*(r['commonsURL373'] or []), *(r['commonsURL3608'] or [])])), axis=1
    )

    # OSM-länkar
    def _merge_osm(r):
        lst = [*(r['osmRelURL'] or []), *(r['osmWayURL'] or []), *(r['osmNodeURL'] or [])]
        seen, out = set(), []
        for u in lst:
            if u not in seen:
                seen.add(u); out.append(u)
        return out
    g['osmLinks'] = g.apply(_merge_osm, axis=1)

    # Flagga operator = SAT (Q131318799)
    g['is_sat_operator'] = g['operator'].apply(lambda ops: any(op == Q_SAT for op in (ops or [])))

    return g
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    """
    Notebook/Edge-säker About-box med string.Template ($-placeholder).
    """
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("SAT Dashboard", "dashboard.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
  @media (max-width: 640px) { .sat-about { font-size: 11px; min-width: 200px; max-width: 260px; } }
</style>

<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">$links_html</div>
  </div>
</div>

<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";

  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }

  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      var willCollapse = !box.classList.contains("sat-about-collapsed");
      setCollapsed(box, willCollapse);
    });

    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }

  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_snippet = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        collapsed_class=collapsed_class,
        issue_url=issue_url,
        issue_number=str(issue_number),
        map_name=_html.escape(map_name),
        created_date=_html.escape(created_date),
        links_html=links_html,
        map_dom_id=map_dom_id,
    )
    m.get_root().html.add_child(folium.Element(html_snippet))

def build_popup_html(row: pd.Series) -> str:
    website = row.get("website")
    if website and "grillplatser.nu" in website:
        website = None
    img_html = f'<div><img src="{row["img"]}" style="max-width:240px; border-radius:8px;"></div>' if row.get("img") else ""

    links: List[str] = []
    if website:
        links.append(f'<a href="{website}" target="_blank" rel="noopener">Webbplats</a>')
    if row.get("naturkartanURL"):
        links.append(f'<a href="{row["naturkartanURL"]}" target="_blank" rel="noopener">Naturkartan</a>')
    for idx, u in enumerate(row.get("commonsURLS") or [], 1):
        label = "Wikimedia Commons" if len(row["commonsURLS"]) == 1 else f"Wikimedia Commons {idx}"
        links.append(f'<a href="{u}" target="_blank" rel="noopener">{label}</a>')
    if row.get("grillplatsnuref"):
        links.append(f'<a href="{row["grillplatsnuref"]}" target="_blank" rel="noopener">grillplatser.nu</a>')
    for idx, u in enumerate(row.get("osmLinks") or [], 1):
        label = "OSM" if len(row["osmLinks"]) == 1 else f"OSM {idx}"
        links.append(f'<a href="{u}" target="_blank" rel="noopener">{label}</a>')

    title = row.get("itemLabel") or "Okänd plats"
    links_html = " · ".join(links) if links else "—"
    return f"<b>{title}</b><br>{img_html}{links_html}"

def add_sat_geojson(m: folium.Map) -> Optional[str]:
    for p in SAT_PATHS:
        if p.exists():
            try:
                gj = folium.GeoJson(p.read_text(encoding="utf-8"), name="SAT",
                    style_function=lambda f: {"weight": 3},
                    highlight_function=lambda f: {"weight": 5})
                gj.add_to(m); return str(p)
            except Exception:
                try:
                    gj = folium.GeoJson(json.loads(p.read_text(encoding="utf-8")), name="SAT")
                    gj.add_to(m); return str(p)
                except Exception: pass
    return None


def pick_icon_and_color(row: pd.Series) -> tuple:
    """Välj ikon + färg.
    - Ikon: grillplats (Q1546788) → FontAwesome 'fire', vindskydd (Q1797440) → 'home'.
    - Färg: blå om operator = Q131318799 (SAT), annars grön om OSM-länk finns, annars röd.
    """
    # Ikon per typ
    t = row.get("type")
    fa_icon = "fire" if t == Q_GRILLPLATS else "home"

    # Färg per operator/OSM
    if row.get("is_sat_operator"):
        color = "blue"
    elif row.get("osmLinks"):
        color = "green"
    else:
        color = "red"
    return fa_icon, color


def main():
    df = run_sparql()
    if df.empty:
        raise SystemExit("Inga resultat från Wikidata.")

    m = folium.Map(location=[df["lat"].mean(), df["lon"].mean()], zoom_start=9, tiles="CartoDB positron")
    add_sat_geojson(m)

    fg_all = folium.FeatureGroup("Vindskydd & grillplatser", show=True)

    for _, row in df.iterrows():
        fa_icon, color = pick_icon_and_color(row)
        marker = folium.Marker(
            [row["lat"], row["lon"]],
            popup=folium.Popup(build_popup_html(row), max_width=320),
            tooltip=row.get("itemLabel"),
            icon=folium.Icon(color=color, icon=fa_icon, prefix='fa')
        )
        fg_all.add_child(marker)

    fg_all.add_to(m)
    folium.LayerControl().add_to(m)
    m.save("map_vindskydd_grillplatser.html")


if __name__ == "__main__":
    main()
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Vindskydd & grillplatser (SAT) → Wikidata → Folium-karta
- OSM-länkar hämtas direkt från Wikidata (Relation: P402, Way: P10689, Node: P11693).
- Hämtar även P137 (operator) och färgkodar markörer som har operator = Q131318799 (Stockholm Archipelago Trail).
- Olika ikoner för grillplats (Q1546788) respektive vindskydd (Q1797440).
- SAT-linjen läses från lokal GeoJSON: SAT_full.geojson (om finns).

Krav:
    pip install pandas folium requests SPARQLWrapper

Kör:
    python vindskydd_grillplatser_map.py

Utdata:
    map_vindskydd_grillplatser.html
"""

import json
from pathlib import Path
from typing import List, Optional

import pandas as pd
import folium
from SPARQLWrapper import SPARQLWrapper, JSON

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

# Sökvägar för SAT-geojson
SAT_PATHS = [Path("SAT_full.geojson"), Path("/mnt/data/SAT_full.geojson")]

# Q-id
Q_GRILLPLATS = "http://www.wikidata.org/entity/Q1546788"
Q_VINDSKYDD  = "http://www.wikidata.org/entity/Q1797440"
Q_SAT        = "http://www.wikidata.org/entity/Q131318799"  # Stockholm Archipelago Trail

SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?website ?naturkartanURL ?commonsURL373 ?commonsURL3608 ?grillplatsnuref
       ?osmRelURL ?osmWayURL ?osmNodeURL ?type ?operator WHERE {
  VALUES ?type { wd:Q1546788 wd:Q1797440 }        # grillplats, vindskydd
  ?item wdt:P31 ?type .
  ?item wdt:P6104 wd:Q134294510 .                 # kopplad till Stockholm Archipelago Trail
  ?item wdt:P625 ?coord .

  OPTIONAL { ?item wdt:P18 ?img }
  OPTIONAL { ?item wdt:P856 ?website }
  OPTIONAL { ?item wdt:P137 ?operator }           # operator

  OPTIONAL {
    ?item wdt:P10467 ?naturkartan .
    BIND( IRI(CONCAT("https://www.naturkartan.se/", STR(?naturkartan))) AS ?naturkartanURL )
  }
  OPTIONAL {
    ?item wdt:P373 ?commons373 .
    BIND( IRI(CONCAT("https://commons.wikimedia.org/wiki/Category:", ENCODE_FOR_URI(STR(?commons373)))) AS ?commonsURL373 )
  }
  OPTIONAL {
    ?item wdt:P3608 ?commons3608 .
    BIND( IRI(CONCAT("https://commons.wikimedia.org/wiki/Category:", ENCODE_FOR_URI(STR(?commons3608)))) AS ?commonsURL3608 )
  }
  OPTIONAL {
    ?item wdt:P1343 wd:Q120778083 .
    ?item p:P1343 ?statement .
    ?statement prov:wasDerivedFrom ?ref .
    ?ref pr:P854 ?grillplatsnuref .
  }

  # OSM-kopplingar (relation, way, node)
  OPTIONAL {
    ?item wdt:P402 ?osmRel .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/relation/", STR(?osmRel))) AS ?osmRelURL )
  }
  OPTIONAL {
    ?item wdt:P10689 ?osmWay .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/way/", STR(?osmWay))) AS ?osmWayURL )
  }
  OPTIONAL {
    ?item wdt:P11693 ?osmNode .
    BIND( IRI(CONCAT("https://www.openstreetmap.org/node/", STR(?osmNode))) AS ?osmNodeURL )
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,mul". }
}
"""

def run_sparql() -> pd.DataFrame:
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT, agent="SAT-vindskydd-map/2.2")
    sparql.setQuery(SPARQL)
    sparql.setReturnFormat(JSON)
    data = sparql.query().convert()
    rows = []
    for b in data["results"]["bindings"]:
        def val(x, f="value"):
            return b[x][f] if x in b else None
        coord = val("coord")
        lat, lon = None, None
        if coord:
            try:
                inside = coord.split("(")[1].split(")")[0].strip()
                parts = inside.split(" ")
                lon = float(parts[0]); lat = float(parts[1])
            except Exception:
                pass
        rows.append({
            "item": val("item"),
            "itemLabel": val("itemLabel"),
            "lat": lat, "lon": lon,
            "img": val("img"),
            "website": val("website"),
            "naturkartanURL": val("naturkartanURL"),
            "commonsURL373": val("commonsURL373"),
            "commonsURL3608": val("commonsURL3608"),
            "grillplatsnuref": val("grillplatsnuref"),
            "osmRelURL": val("osmRelURL"),
            "osmWayURL": val("osmWayURL"),
            "osmNodeURL": val("osmNodeURL"),
            "type": val("type"),
            "operator": val("operator"),
        })
    df = pd.DataFrame(rows)
    if df.empty:
        return df

    agg = {
        'itemLabel': 'first', 'lat': 'first', 'lon': 'first',
        'img': 'first', 'website': 'first', 'naturkartanURL': 'first', 'grillplatsnuref': 'first',
        'commonsURL373': lambda s: list({x for x in s if pd.notna(x)}),
        'commonsURL3608': lambda s: list({x for x in s if pd.notna(x)}),
        'osmRelURL': lambda s: list({x for x in s if pd.notna(x)}),
        'osmWayURL': lambda s: list({x for x in s if pd.notna(x)}),
        'osmNodeURL': lambda s: list({x for x in s if pd.notna(x)}),
        'type': 'first',
        'operator': lambda s: list({x for x in s if pd.notna(x)}),
    }
    g = df.groupby('item', as_index=False).agg(agg)

    # Commons-länkar
    g['commonsURLS'] = g.apply(
        lambda r: list(dict.fromkeys([*(r['commonsURL373'] or []), *(r['commonsURL3608'] or [])])), axis=1
    )

    # OSM-länkar
    def _merge_osm(r):
        lst = [*(r['osmRelURL'] or []), *(r['osmWayURL'] or []), *(r['osmNodeURL'] or [])]
        seen, out = set(), []
        for u in lst:
            if u not in seen:
                seen.add(u); out.append(u)
        return out
    g['osmLinks'] = g.apply(_merge_osm, axis=1)

    # Flagga operator = SAT (Q131318799)
    g['is_sat_operator'] = g['operator'].apply(lambda ops: any(op == Q_SAT for op in (ops or [])))

    return g

def build_popup_html(row: pd.Series) -> str:
    website = row.get("website")
    if website and "grillplatser.nu" in website:
        website = None
    img_html = f'<div><img src="{row["img"]}" style="max-width:240px; border-radius:8px;"></div>' if row.get("img") else ""

    links: List[str] = []
    if website:
        links.append(f'<a href="{website}" target="_blank" rel="noopener">Webbplats</a>')
    if row.get("naturkartanURL"):
        links.append(f'<a href="{row["naturkartanURL"]}" target="_blank" rel="noopener">Naturkartan</a>')
    for idx, u in enumerate(row.get("commonsURLS") or [], 1):
        label = "Wikimedia Commons" if len(row["commonsURLS"]) == 1 else f"Wikimedia Commons {idx}"
        links.append(f'<a href="{u}" target="_blank" rel="noopener">{label}</a>')
    if row.get("grillplatsnuref"):
        links.append(f'<a href="{row["grillplatsnuref"]}" target="_blank" rel="noopener">grillplatser.nu</a>')
    for idx, u in enumerate(row.get("osmLinks") or [], 1):
        label = "OSM" if len(row["osmLinks"]) == 1 else f"OSM {idx}"
        links.append(f'<a href="{u}" target="_blank" rel="noopener">{label}</a>')

    title = row.get("itemLabel") or "Okänd plats"
    links_html = " · ".join(links) if links else "—"
    return f"<b>{title}</b><br>{img_html}{links_html}"

def add_sat_geojson(m: folium.Map) -> Optional[str]:
    for p in SAT_PATHS:
        if p.exists():
            try:
                gj = folium.GeoJson(p.read_text(encoding="utf-8"), name="SAT",
                    style_function=lambda f: {"weight": 3},
                    highlight_function=lambda f: {"weight": 5})
                gj.add_to(m); return str(p)
            except Exception:
                try:
                    gj = folium.GeoJson(json.loads(p.read_text(encoding="utf-8")), name="SAT")
                    gj.add_to(m); return str(p)
                except Exception: pass
    return None


def pick_icon_and_color(row: pd.Series) -> tuple:
    """Välj ikon + färg.
    - Ikon: grillplats (Q1546788) → FontAwesome 'fire', vindskydd (Q1797440) → 'home'.
    - Färg: blå om operator = Q131318799 (SAT), annars grön om OSM-länk finns, annars röd.
    """
    # Ikon per typ
    t = row.get("type")
    fa_icon = "fire" if t == Q_GRILLPLATS else "home"

    # Färg per operator/OSM
    if row.get("is_sat_operator"):
        color = "blue"
    elif row.get("osmLinks"):
        color = "green"
    else:
        color = "red"
    return fa_icon, color


def main():
    df = run_sparql()
    if df.empty:
        raise SystemExit("Inga resultat från Wikidata.")

    m = folium.Map(location=[df["lat"].mean(), df["lon"].mean()], zoom_start=9, tiles="CartoDB positron")
    add_sat_geojson(m)

    fg_all = folium.FeatureGroup("Vindskydd & grillplatser", show=True)

    for _, row in df.iterrows():
        fa_icon, color = pick_icon_and_color(row)
        marker = folium.Marker(
            [row["lat"], row["lon"]],
            popup=folium.Popup(build_popup_html(row), max_width=320),
            tooltip=row.get("itemLabel"),
            icon=folium.Icon(color=color, icon=fa_icon, prefix='fa')
        )
        fg_all.add_child(marker)
    add_about_box(m, issue_number=185, map_name="Vindskydd grillplatser")

    fg_all.add_to(m)
    folium.LayerControl().add_to(m)
    filename = "output/map185_vindskydd_grillplatser.html"
    m.save(filename)
    print("Map created " + filename)

if __name__ == "__main__":
    main()


Map created output/map185_vindskydd_grillplatser.html


In [ ]:
end_time = time.time()
duration = end_time - start_time
print(f"Finished in {duration:.2f} seconds.")
